# 3-qubit teleportation with configurable noise

This notebook mirrors the feed-forward teleportation circuit and introduces realistic single- and two-qubit gate errors plus readout noise. Adjust the parameters below to explore how fidelity degrades under different noise strengths.


In [5]:
from qiskit import QuantumCircuit, ClassicalRegister, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError


In [6]:
def build_noise_model(p_single=1e-3, p_two=5e-3, p_readout=2e-2):
    """Return a Qiskit NoiseModel with depolarizing gate noise and readout errors."""
    noise_model = NoiseModel()

    single_error = depolarizing_error(p_single, 1)
    two_error = depolarizing_error(p_two, 2)
    readout_error = ReadoutError([[1 - p_readout, p_readout], [p_readout, 1 - p_readout]])

    single_gate_set = ["id", "x", "sx", "rz", "h"]
    two_gate_set = ["cx", "cz"]

    noise_model.add_all_qubit_quantum_error(single_error, single_gate_set)
    noise_model.add_all_qubit_quantum_error(two_error, two_gate_set)
    noise_model.add_all_qubit_readout_error(readout_error)
    return noise_model


In [9]:
def original_pair_circuit():
    circ = QuantumCircuit(3)
    circ.h(0)
    circ.cx(0, 1)
    circ.save_density_matrix(label="rho_orig")
    return circ


def teleportation_circuit():
    tele_circ = QuantumCircuit(3, 1)
    tele_circ.h(0)
    tele_circ.cx(0, 1)
    tele_circ.h(2)
    tele_circ.cz(1, 2)
    tele_circ.h(1)
    tele_circ.measure(1, 0)
    with tele_circ.if_test((tele_circ.cregs[0], 1)):
        tele_circ.x(2)
    tele_circ.h(2)
    tele_circ.save_density_matrix(label="rho_final")
    return tele_circ


def simulate_density_matrix(circuit, simulator, label):
    compiled = transpile(circuit, simulator)
    result = simulator.run(compiled).result()
    return DensityMatrix(result.data(0)[label])


In [10]:
noise_params = {
    "p_single": 2e-3,
    "p_two": 1e-2,
    "p_readout": 1.5e-2,
}

ideal_sim = AerSimulator(method="density_matrix")
noise_model = build_noise_model(**noise_params)
noisy_sim = AerSimulator(method="density_matrix", noise_model=noise_model)

rho_full_orig = simulate_density_matrix(original_pair_circuit(), ideal_sim, "rho_orig")
rho_01 = partial_trace(rho_full_orig, [2])

rho_full_tele_ideal = simulate_density_matrix(teleportation_circuit(), ideal_sim, "rho_final")
rho_02_ideal = partial_trace(rho_full_tele_ideal, [1])

rho_full_tele_noisy = simulate_density_matrix(teleportation_circuit(), noisy_sim, "rho_final")
rho_02_noisy = partial_trace(rho_full_tele_noisy, [1])

print("Noise parameters:", noise_params)
print("Fidelity (ideal teleportation vs original pair):", state_fidelity(rho_01, rho_02_ideal))
print("Fidelity (noisy teleportation vs original pair):", state_fidelity(rho_01, rho_02_noisy))


Noise parameters: {'p_single': 0.002, 'p_two': 0.01, 'p_readout': 0.015}
Fidelity (ideal teleportation vs original pair): 1.0000000000000004
Fidelity (noisy teleportation vs original pair): 0.9694930340966663


In [11]:
scan_points = [5e-4, 2e-3, 1e-2]
print("\nSingle-qubit depolarizing scan:")
for p_single in scan_points:
    params = {"p_single": p_single, "p_two": 5 * p_single, "p_readout": noise_params["p_readout"]}
    nm = build_noise_model(**params)
    sim = AerSimulator(method="density_matrix", noise_model=nm)
    rho_tele = partial_trace(simulate_density_matrix(teleportation_circuit(), sim, "rho_final"), [1])
    fidelity = state_fidelity(rho_01, rho_tele)
    print(f"  p_single={p_single:.1e}, p_two={5 * p_single:.1e} -> fidelity={fidelity:.6f}")



Single-qubit depolarizing scan:
  p_single=5.0e-04, p_two=2.5e-03 -> fidelity=0.979424
  p_single=2.0e-03, p_two=1.0e-02 -> fidelity=0.968545
  p_single=1.0e-02, p_two=5.0e-02 -> fidelity=0.890968
